In [1]:
import sys
import os

# Add the directory root in the path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from utils.extract_text import extract_text_from_svg

complete_text = extract_text_from_svg()

In [3]:
for i,text in enumerate(complete_text):
    if i < 6:
        print(text)


1º Tesoureiro
Luiz Henrique Mendes Costa

Semestre: 2º
Gosta de:
   animes
manhwa
games
RPG
cibersegurança
curiosidades
      tecnológicas

henrique@ubuntu 


In [4]:
len(complete_text)

164

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=10,
    separators=["\n\n", "\n", " ", ""],
)

chunks = text_splitter.create_documents(complete_text)

In [6]:
print(f'number of chunks: {len(chunks)}')

for i,chunk in enumerate(chunks):
    if i < 6:
        print(f'Chunk {i+1}: len={len(chunk.page_content)}')
        print(chunk.page_content)

number of chunks: 115
Chunk 1: len=13
1º Tesoureiro
Chunk 2: len=26
Luiz Henrique Mendes Costa
Chunk 3: len=96
Semestre: 2º
Gosta de:
   animes
manhwa
games
RPG
cibersegurança
curiosidades
      tecnológicas
Chunk 4: len=15
henrique@ubuntu
Chunk 5: len=13
1º Secretário
Chunk 6: len=24
João Davi Costa de Souza


In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

model_name = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device":"cpu"}
)

vector_store = Chroma.from_documents(
    documents=chunks, 
    embedding=embeddings,
    persist_directory="../chroma_db"
)

retriever = vector_store.as_retriever()

In [8]:
query = "What kind of environment does Yuri use?"

relevant_docs = retriever.invoke(query)

print(f"\n '{query}'")

for i, doc in enumerate(relevant_docs):
    if i < 6:
        print(f"Document {i+1} (Context):")
        print(doc.page_content)


 'What kind of environment does Yuri use?'
Document 1 (Context):
Yuri Gabriel Cardoso Delgado
Document 2 (Context):
Semestre: 2º
Gosta de:
   Tecnologia
Estudar
Séries
Filmes
Jogos
Desenhos
Environment:
Hyprland (WM)
Kitty (terminal)
VS Code (Editor)
Document 3 (Context):
O Tesoureiro Geral, Yuri Delgado, e o Secretário Geral, Leonardo Brito, apesar dos compromissos e de outros dificultantes, também demonstraram interesse em participar.
Document 4 (Context):
Semestre: 2º
Gosta de:
Linux
Ficção Científica
Low-Level
KPOP
TRON
Environment:
Hyprland (WM)
ST (terminal)
Neovim (Editor)


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

from langchain_ollama import OllamaLLM
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

#llm = OllamaLLM(model="llama3")

prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant. Use the following context to answer the question.
If you don't know the answer, say that you don't have enough information in the context.

Contexto:
{context}

Pergunta: {input}
""")

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

rag_chain = create_retrieval_chain(retriever, document_chain)

query = "Tell me What kind of environment does Lucas use?."

response = rag_chain.invoke({"input": query})

print(f"\nresponse: {response['answer']}")


response: Lucas uses the following environment:
*   **WM:** Hyprland
*   **Terminal:** Kitty
*   **Editor:** VS Code
